In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.linalg as linalg
from numba import jit

In [2]:
from scipy.stats import norm
def Black_Scholes_Call(S, K, r, vol, tau):
    """ 
    Black Scholes Model for European Call
    """
    d1 = (np.log(S / K) + (r + ((vol**2)/2.)*tau)) / (vol*np.sqrt(tau))
    d2 = d1 - vol*np.sqrt(tau)
    V = S * norm.cdf(d1) - np.exp(-r*tau) * K * norm.cdf(d2)
    
    return V

In [31]:
sigma = 0.3
K = 110
S = 100
r = 0.04
T = 1
N = 400# time points
M =  400# space points

In [32]:
Black_Scholes_Call(S, K, r, sigma, T)

9.625357828843697

In [33]:
FTCS(S, K, r, sigma, T, N, M)

0.0025
0.018
-0.3475694444444445 1.6945444444444446 -0.3468750000000001
[[-1.          1.          0.         ...  0.          0.
   0.        ]
 [-0.34756944  1.69454444 -0.346875   ...  0.          0.
   0.        ]
 [ 0.         -0.34756944  1.69454444 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  1.69454444 -0.346875
   0.        ]
 [ 0.          0.          0.         ... -0.34756944  1.69454444
  -0.346875  ]
 [ 0.          0.          0.         ...  0.         -1.
   1.        ]]


9.630705997071836

In [14]:
def FTCS(S, K, r, vol, T, N, M):
    # Compute delta T
    dt = T/N
    print(dt)

    # Assignmen 3 part 3 boundaries
    x_max = np.log(S) + 12*sigma*np.sqrt(T);
    x_min = np.log(S) - 12*sigma*np.sqrt(T);
    dx = (x_max - x_min)/M;
    print(dx)

    # Generate stock prices on log scale
    X = np.linspace(x_min, x_max, M+1)
    # Generate T * S grid
    V = np.zeros((N+1, M+1))

    # Put the discounted values into the first column
    V[0, :] = [max(np.exp(stocks)-K,0) for stocks in X]

    # Fucking constants
    ss = sigma * sigma
    dxx = dx*dx
    
    # a-1, a0, a+1
    ad = (dt/2)* ((r - 1/2*ss)/dx - ss/dxx)
    a0 = 1 + dt*(ss/(dxx) + r)
    au = (dt/2)*(-ss/dxx - (r - 1/2*ss)/dx)
    print(ad, a0, au)

    # Generate matrix A
    A = np.zeros((M+1, M+1))
    for i in range(1, M):
        A[i,i-1] = ad
        A[i,i] = a0
        A[i,i+1] = au
    A[0,0], A[0,1], A[-1,-1], A[-1,-2] = -1, 1, 1, -1
    #A[0,0], A[0,1], A[-1,-1], A[-1,-2] = 0,0,0,0
    print(A)

    # V[n+1] = A * V[m]
    for i in range(0,N):
        V[i+1, :] = np.linalg.solve(A,V[i,:])

    return V[-1,int(M/2)]

In [ ]:
def FTCS(S, K, r, vol, T, N, M):
    # Compute delta T
    dt = T/N
    print(dt)

    # Assignmen 3 part 3 boundaries
    x_max = np.log(S) + 12*sigma*np.sqrt(T);
    x_min = np.log(S) - 12*sigma*np.sqrt(T);
    dx = (x_max - x_min)/M;
    print(dx)

    # Generate stock prices on log scale
    X = np.linspace(x_min, x_max, M+1)
    # Generate T * S grid
    V = np.zeros((N+1, M+1))

    # Put the discounted values into the first column
    V[0, :] = [max(np.exp(stocks)-K,0) for stocks in X]

    # Fucking constants
    ss = sigma * sigma
    dxx = dx*dx
    
    # a-1, a0, a+1
    ad = 0.5 * (ss * dxx * dt - r*dxx*dt)
    a0 = 1 + dt*(ss/(dxx) + r)
    au = (dt/2)*(-ss/dxx - (r - 1/2*ss)/dx)
    print(ad, a0, au)

    # Generate matrix A
    A = np.zeros((M+1, M+1))
    for i in range(1, M):
        A[i,i-1] = ad
        A[i,i] = a0
        A[i,i+1] = au
    A[0,0], A[0,1], A[-1,-1], A[-1,-2] = -1, 1, 1, -1
    #A[0,0], A[0,1], A[-1,-1], A[-1,-2] = 0,0,0,0
    print(A)

    # V[n+1] = A * V[m]
    for i in range(0,N):
        V[i+1, :] = np.linalg.solve(A,V[i,:])

    return V[-1,int(M/2)]